In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import sys
os.chdir('/content/gdrive/My Drive/MIT/Research/Notebooks/EnrichedTiSASRecV7-Pytorch')
sys.path.append("/content/gdrive/My Drive/MIT/Research/Notebooks/EnrichedTiSASRecV7-Pytorch")
!pwd

/content/gdrive/My Drive/MIT/Research/Notebooks/EnrichedTiSASRecV7-Pytorch


In [3]:
!ls

 data					 inference.py   README.md
'EnrichedTiSASRecNotes - V7.txt'	 main.py        train_enriched_tisasrec_v7.ipynb
 EnrichedTiSASRec_test_functions.ipynb	 model.py       utils.py
 EnrichedTiSASRec_test_model.ipynb	 __pycache__


In [4]:
from itertools import islice

def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

In [5]:
import os
import torch
import pickle
import argparse

from model import EnrichedTiSASRec

def str2bool(s):
    if s not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s == 'true'

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='BooksWithCategoryPercentage', type=str)
parser.add_argument('--train_dir', default='default', type=str)
parser.add_argument('--batch_size', default=1, type=int)
parser.add_argument('--lr', default=0.001, type=float)
parser.add_argument('--maxlen', default=5, type=int)
parser.add_argument('--hidden_units', default=64, type=int)
parser.add_argument('--num_blocks', default=2, type=int)
parser.add_argument('--num_epochs', default=10, type=int)
parser.add_argument('--num_heads', default=1, type=int)
parser.add_argument('--dropout_rate', default=0.2, type=float)
parser.add_argument('--l2_emb', default=0.00005, type=float)
parser.add_argument('--device', default='cpu', type=str)
parser.add_argument('--inference_only', default=False, type=str2bool)
parser.add_argument('--state_dict_path', default=None, type=str)
parser.add_argument('--time_span', default=256, type=int)

args = parser.parse_args(args=[])
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
from utils import data_partition

dataset = data_partition(args.dataset)
[user_train, user_valid, user_test, usernum, itemnum, timenum, catnum] = dataset

print(take(1, user_train.items()))

Preparing data...
Preparing done...
[(1, [[13788, 1, 4, 9], [23426, 10, 5, 9], [11079, 12, 4, 9], [14643, 12, 5, 28], [38738, 12, 5, 9], [29945, 17, 5, 28], [16515, 17, 5, 28], [37090, 26, 4, 28], [10217, 37, 5, 4], [29256, 37, 5, 9], [14055, 37, 5, 9], [47793, 37, 5, 0], [29477, 40, 5, 1], [43344, 40, 5, 1], [42244, 43, 4, 0], [31694, 47, 5, 0], [40127, 69, 5, 9], [47738, 70, 4, 28], [14912, 82, 5, 28], [7986, 82, 5, 0], [26180, 94, 5, 28], [39976, 106, 5, 9], [20768, 110, 5, 0], [5254, 124, 5, 28], [47813, 124, 5, 28], [1800, 124, 5, 28], [17503, 127, 5, 9], [15333, 127, 5, 9], [38697, 128, 5, 28], [25387, 142, 5, 28]])]


In [7]:
cc = 0.0
len_dict = dict()
for u in user_train:
  len_u = len(user_train[u])
  len_dict[u] = len_u
  cc += len_u
print('average sequence length: %.2f' % (cc / len(user_train)))

average sequence length: 75.13


In [8]:
num_batch = len(user_train) // args.batch_size
num_batch

57208

In [9]:
from utils import WarpSampler, Relation
import numpy as np

try:
    relation_matrix = pickle.load(open('data/relation_matrix_%s_%d_%d.pickle'%(args.dataset, args.maxlen, args.time_span),'rb'))
except:
    relation_matrix = Relation(user_train, usernum, args.maxlen, args.time_span)
    pickle.dump(relation_matrix, open('data/relation_matrix_%s_%d_%d.pickle'%(args.dataset, args.maxlen, args.time_span),'wb'))

sampler = WarpSampler(user_train, usernum, itemnum, relation_matrix, batch_size=args.batch_size, maxlen=args.maxlen, n_workers=3)

u, seq, rat_seq, time_seq, time_matrix, cat_seq, pos, neg = sampler.next_batch() # tuples to ndarray


In [10]:
u, seq, rat_seq, time_seq, time_matrix, cat_seq, pos, neg = sampler.next_batch() # tuples to ndarray

u, seq, rat_seq, cat_seq, pos, neg = np.array(u), np.array(seq), np.array(rat_seq), np.array(cat_seq), np.array(pos), np.array(neg)

time_seq, time_matrix = np.array(time_seq), np.array(time_matrix)

print("-----------------User----------------------")
print(u)
print("-----------------Seq-----------------------")
print(seq)
print(np.shape(seq))
print("-----------------Rating Seq-----------------------")
print(rat_seq)
print(np.shape(seq))
print("-----------------Category Seq-----------------------")
print(cat_seq)
print(np.shape(rat_seq))
print("-----------------Pos----------------------")
print(pos)
print(np.shape(pos))
print("-----------------Neg----------------------")
print(neg)
print(np.shape(neg))
print("-----------------Time Seq----------------------")
print(time_seq)
print(np.shape(time_seq))
print("-----------------Time Matrix----------------------")
print(time_matrix)
print(np.shape(time_matrix))

-----------------User----------------------
[23793]
-----------------Seq-----------------------
[[ 6548 23694 16674 11366 13620]]
(1, 5)
-----------------Rating Seq-----------------------
[[5 5 5 5 5]]
(1, 5)
-----------------Category Seq-----------------------
[[1 6 9 9 1]]
(1, 5)
-----------------Pos----------------------
[[23694 16674 11366 13620 22666]]
(1, 5)
-----------------Neg----------------------
[[48162 30027 31312 33937 22453]]
(1, 5)
-----------------Time Seq----------------------
[[473 543 571 633 640]]
(1, 5)
-----------------Time Matrix----------------------
[[[  0  70  98 160 167]
  [ 70   0  28  90  97]
  [ 98  28   0  62  69]
  [160  90  62   0   7]
  [167  97  69   7   0]]]
(1, 5, 5)


In [11]:
print(len(user_train[u[0]]))
user_train[u[0]]

43


[[30570, 1, 5, 28],
 [18958, 20, 5, 6],
 [48800, 58, 5, 4],
 [899, 65, 5, 0],
 [1490, 69, 5, 28],
 [33357, 84, 4, 9],
 [862, 86, 5, 9],
 [32278, 92, 5, 28],
 [19270, 103, 4, 28],
 [40779, 105, 5, 28],
 [8889, 109, 5, 28],
 [19960, 111, 4, 6],
 [25212, 115, 5, 4],
 [10547, 117, 4, 6],
 [7691, 119, 4, 9],
 [39160, 123, 5, 0],
 [44371, 127, 5, 0],
 [32743, 127, 4, 3],
 [15490, 136, 4, 28],
 [28738, 140, 5, 6],
 [30977, 148, 5, 9],
 [37933, 149, 5, 4],
 [1286, 153, 5, 28],
 [39642, 154, 5, 28],
 [36397, 157, 5, 28],
 [23081, 159, 5, 28],
 [43364, 162, 4, 3],
 [9673, 171, 4, 28],
 [11448, 172, 5, 9],
 [43696, 176, 5, 4],
 [28996, 184, 5, 28],
 [33850, 187, 4, 4],
 [20241, 189, 5, 28],
 [30184, 203, 5, 28],
 [8425, 415, 4, 6],
 [36769, 430, 5, 4],
 [46471, 462, 5, 5],
 [6548, 473, 5, 1],
 [23694, 543, 5, 6],
 [16674, 571, 5, 9],
 [11366, 633, 5, 9],
 [13620, 640, 5, 1],
 [22666, 652, 5, 28]]

In [12]:
!ls

 data					 inference.py   README.md
'EnrichedTiSASRecNotes - V7.txt'	 main.py        train_enriched_tisasrec_v7.ipynb
 EnrichedTiSASRec_test_functions.ipynb	 model.py       utils.py
 EnrichedTiSASRec_test_model.ipynb	 __pycache__


### Test model

In [13]:
model = EnrichedTiSASRec(usernum, itemnum, timenum, catnum, args).to(args.device)
model

EnrichedTiSASRec(
  (item_emb): Embedding(51023, 64, padding_idx=0)
  (item_emb_dropout): Dropout(p=0.2, inplace=False)
  (rat_K_emb): Embedding(6, 64)
  (rat_V_emb): Embedding(6, 64)
  (cat_K_emb): Embedding(100, 64)
  (cat_V_emb): Embedding(100, 64)
  (abs_pos_K_emb): Embedding(5, 64)
  (abs_pos_V_emb): Embedding(5, 64)
  (time_matrix_K_emb): Embedding(257, 64)
  (time_matrix_V_emb): Embedding(257, 64)
  (abs_pos_K_emb_dropout): Dropout(p=0.2, inplace=False)
  (abs_pos_V_emb_dropout): Dropout(p=0.2, inplace=False)
  (time_matrix_K_dropout): Dropout(p=0.2, inplace=False)
  (time_matrix_V_dropout): Dropout(p=0.2, inplace=False)
  (rat_K_emb_dropout): Dropout(p=0.2, inplace=False)
  (rat_V_emb_dropout): Dropout(p=0.2, inplace=False)
  (cat_K_emb_dropout): Dropout(p=0.2, inplace=False)
  (cat_V_emb_dropout): Dropout(p=0.2, inplace=False)
  (attention_layernorms): ModuleList(
    (0-1): 2 x LayerNorm((64,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0-1

In [14]:
pos_logits, neg_logits = model(u, seq, rat_seq, cat_seq, time_matrix, pos, neg)
pos_logits

tensor([[-0.5365,  7.7019,  6.6572, 19.2305, 12.9291]], grad_fn=<SumBackward1>)

In [15]:
pos_labels, neg_labels = torch.ones(pos_logits.shape, device=args.device), torch.zeros(neg_logits.shape, device=args.device)
pos_labels, neg_labels

(tensor([[1., 1., 1., 1., 1.]]), tensor([[0., 0., 0., 0., 0.]]))

In [ ]:
!python inference.py

['1250048133', '0425276031', '1514210665', '0470580461', '1501142275', '1617737747', '0307732177', '1983998648', '1455539767', '0373004559', '1478244399', '1300773375']
